In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser

In [2]:
# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

In [3]:
# extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        info.append([email.get("From"), email.get('To'), email.get('cc'), email.get('bcc'), 
                     str(email.get("Content-Type")).split(";")[0],
                     email.get('subject'), " ".join([i for i in walkMsg(email) if i != None]),classification])
    return info

In [4]:
listing = os.listdir('./Ham_BG/')
listing = [i for i in listing if i[-3:] == "txt"]
email_df = pd.DataFrame(info_of_email('./Ham_BG/', listing, 'spam'),
                       columns=['From','To','cc','bcc','Content_type','Subjects','Content','Classification'])

In [5]:
email_df.head()

,From,To,cc,bcc,Content_type,Subjects,Content,Classification
0,"""Bertha "" <Denny@mailbox.sk>",rait@bruce-guenter.dyndns.org,None,None,multipart/alternative,Get it up and keep it up praecox,<html><head><meta http-equiv=Content-Type cont...,spam
1,"""LZEDEMCNODQBEZQBOV@encryption.com"" <Opal.Cham...",cvs@bruce-guenter.dyndns.org,rait@bruce-guenter.dyndns.org,None,multipart/alternative,Winner: $332573,<html>\nYour [m]ortgage application was approv...,spam
2,Connie Wong <tatypin@huhmail.com>,Rait <rait@bruce-guenter.dyndns.org>,None,None,multipart/alternative,Need software? Click here.,TOP quality software:<br><br>\n<b>Special Offe...,spam
3,shark@promotions.com <shark@promotions.com>,"""bruceg@em.ca"" <bruceg@em.ca>",None,None,multipart/alternative,Loyalty Bonuses for Shark Casino Players!,\n \n \n \n \n \n Featuring this weeken...,spam
4,"""Important! Search Engine Alert!"" <rockwelldat...","""Bruceg"" <bruceg@em.ca>",None,None,multipart/alternative,Your web site has dropped out of the Search En...,Rockwell Data Corp \n\nThis Email has been sen...,spam
